In [17]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import csv

Connect to Kafka

In [18]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

Start Streaming

In [19]:
def make_serializable(d):
    serializable_dict = {}
    for key, value in d.items():
        if isinstance(value, (list, dict, str, int, float, bool, type(None))):
            serializable_dict[key] = value
        else:
            # Convert non-serializable value to a serializable type
            serializable_dict[key] = str(value)  # Or perform a suitable conversion
    return serializable_dict

In [28]:
# import os
# MY_SQL_API_URL=os.getenv('MY_SQL_API_URL')
# or
# MY_SQL_API_URL='http://your-MySQL-api-url/endpoint'

SLEEP = 3

def make_serializable(d):
    serializable_dict = {}
    for key, value in d.items():
        if isinstance(value, (list, dict, str, int, float, bool, type(None))):
            serializable_dict[key] = value
        else:
            serializable_dict[key] = str(value)  # Convert non-serializable value to string
    return serializable_dict

SLEEP = 3

def publish_to_kafka(file_path, producer=producer, chunk_size=1000):
    """ 
    Send each row to MySQl API and Kafka.
    
    Uncomment either the environment code if using .env file or 
    variable code to set directly.
    
    SLEEP variable can be adjusted as desired. 1 is the minimum recommended setting. 
    Setting to 0 will crash the Kafka server unless a more robust EC2 setup is created.
    """
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        while True:
            chunk = []
            for _ in range(chunk_size):
                try:
                    chunk.append(next(reader))
                except StopIteration:
                    break
            if not chunk:
                break
            for row in chunk:
                # Convert row to JSON
                row = make_serializable(row)
                json_data = dumps(row)

                # Publish row to Kafka
                producer.send('mysql', value=json_data)
                sleep(SLEEP)
            producer.flush()
    producer.close()

In [29]:
publish_to_kafka("data/Iowa_Liquor_Sales.csv")

KeyboardInterrupt: 

View Topics

Kafka Docker

1. `docker exec -it kafka1 bash`
1. `cd /usr/bin`
1. `kafka-topics --list --bootstrap-server localhost:9092`
1. `kafka-console-consumer --bootstrap-server localhost:9092 --topic mysql --from-beginning`

Manually flush data when done

In [ ]:
producer.flush()

Test

In [ ]:
import pandas as pd

df = pd.read_csv("../data/Iowa_Liquor_Sales.csv", nrows=10)

print(df)

In [ ]:
print(df.columns)

In [ ]:
# sample output
'''
{'Invoice/Item Number': 'S29198800001', 'Date': '11/20/2015', 'Store Number': '2191', 'Store Name': 'Keokuk Spirits', 
'Address': '1013 MAIN', 'City': 'KEOKUK', 'Zip Code': '52632', 'Store Location': '1013 MAIN\nKEOKUK 52632\n(40.39978, -91.387531)', 
'County Number': '56', 'County': 'Lee', 'Category': '', 'Category Name': '', 'Vendor Number': '255', 'Vendor Name': 'Wilson Daniels Ltd.', 
'Item Number': '297', 'Item Description': 'Templeton Rye w/Flask', 'Pack': '6', 'Bottle Volume (ml)': '750', 'State Bottle Cost': '$18.09', 
'State Bottle Retail': '$27.14', 'Bottles Sold': '6', 'Sale (Dollars)': '$162.84', 'Volume Sold (Liters)': '4.50', 'Volume Sold (Gallons)': '1.19'}
'''

In [ ]:
with open("../data/Iowa_Liquor_Sales.csv", 'r') as file:
    reader = csv.DictReader(file)
    while True:
        chunk = []
        for _ in range(1000):
            try:
                chunk.append(next(reader))
            except StopIteration:
                break
        if not chunk:
            break
        for row in chunk:
            print(row)